# Room for the river: Gorssel 

## Imports 

In [1]:
#imports
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import sys

from ema_workbench import (Model, CategoricalParameter,ScalarOutcome, IntegerParameter, RealParameter,
                           MultiprocessingEvaluator, ema_logging, Constant, Policy, Scenario,
                           perform_experiments, SequentialEvaluator,Constraint)
from problem_formulation import get_model_for_problem_formulation

from dike_model_function import DikeNetwork  # @UnresolvedImport

from ema_workbench.em_framework.optimization import (HyperVolume, EpsilonProgress,GenerationalBorg)
from ema_workbench.em_framework.evaluators import (perform_experiments,BaseEvaluator)
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
from ema_workbench.analysis import plotting, plotting_util, parcoords, feature_scoring, prim
from ema_workbench import load_results 

from mpl_toolkits.mplot3d import Axes3D 

from sklearn import preprocessing

import time
import copy

import itertools

from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor
import os
import functools

import random

In [2]:
from scipy.spatial.distance import pdist, squareform

def evaluate_diversity_single(indices, distances, weight=0.5, distance='euclidean'):
    '''
    takes the outcomes and selected scenario set (decision variables), 
    returns a single 'diversity' value for the scenario set.
    outcomes : outcomes dictionary of the scenario ensemble
    decision vars : indices of the scenario set
    weight : weight given to the mean in the diversity metric. If 0, only minimum; if 1, only mean
    '''
    i, j = [e for e in zip(*itertools.combinations(indices, 2))]
    subset_distances = distances[i, j]
    minimum = np.min(subset_distances)
    mean = np.mean(subset_distances)
    diversity = (1-weight)*minimum + weight*mean
    
    return [diversity]


def find_maxdiverse_scenarios(distances, combinations):
    scores = []
    for indices in combinations:
        diversity = evaluate_diversity_single(indices, distances)
        scores.append((diversity, indices))

    return scores

def optimize(scenario, nfe, model, converge_metrics, epsilons):


    with MultiprocessingEvaluator(model) as evaluator:
        results, convergence = evaluator.optimize(nfe=nfe, searchover='levers',
                                     convergence=convergence_metrics,
                                     epsilons=epsilons,
                                     reference=scenario)
    return results, convergence


## Create Data 

### Run Model for outcomes and experiments 

In [3]:
#running the model through EMA workbench
ema_logging.log_to_stderr(ema_logging.INFO)

#choose problem formulation number, between 0-5
#each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(3) 


In [4]:
with SequentialEvaluator(dike_model) as evaluator:
    results_MORDM = evaluator.perform_experiments(scenarios=10, policies=5) 
    #s for number of scenarios, p for number of experiments 
    
experiments_MORDM, outcomes_MORDM = results_MORDM 
#s for number of scenarios, p for number of experiments 

[MainProcess/INFO] performing 10 scenarios * 5 policies * 1 model(s) = 50 experiments
100%|██████████████████████████████████████████| 50/50 [01:03<00:00,  1.28s/it]
[MainProcess/INFO] experiments finished


In [5]:
#save the initial results to be sure the original can be reused without having to run the model again 
raw_outcomes    = outcomes_MORDM.copy()
raw_experiments = experiments_MORDM.copy()

In [6]:
#create dataframes from the dictionaries 

df_experiments = pd.DataFrame(experiments_MORDM)

df_outcomes =pd.DataFrame()
for outcome in outcomes_MORDM.keys():
    df_outcomes[outcome] = outcomes_MORDM[outcome]

df_experiments
df_experiments.columns

Index(['A.0_ID flood wave shape', 'A.1_Bmax', 'A.1_Brate', 'A.1_pfail',
       'A.2_Bmax', 'A.2_Brate', 'A.2_pfail', 'A.3_Bmax', 'A.3_Brate',
       'A.3_pfail', 'A.4_Bmax', 'A.4_Brate', 'A.4_pfail', 'A.5_Bmax',
       'A.5_Brate', 'A.5_pfail', 'discount rate 0', 'discount rate 1',
       'discount rate 2', '0_RfR 0', '0_RfR 1', '0_RfR 2', '1_RfR 0',
       '1_RfR 1', '1_RfR 2', '2_RfR 0', '2_RfR 1', '2_RfR 2', '3_RfR 0',
       '3_RfR 1', '3_RfR 2', '4_RfR 0', '4_RfR 1', '4_RfR 2',
       'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2',
       'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2',
       'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2',
       'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2',
       'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2',
       'EWS_DaysToThreat', 'scenario', 'policy', 'model'],
      dtype='object')

### Load in experiments and outcomes instead of running in this notebook 

In [7]:
df_experiments_s1000_p10 = pd.read_csv("df_experiments_s1000_p10.csv")
df_outcomes_s1000_p10 = pd.read_csv("df_outcomes_s1000_p10.csv")

### Set experiments and outcomes 

In [8]:
experiments = df_experiments#_s1000_p10.copy()
outcomes = df_outcomes#_s1000_p10.copy()

In [9]:
outcomes.keys()

Index(['A.1_Expected Annual Damage', 'A.1_Dike Investment Costs',
       'A.1_Expected Number of Deaths', 'A.2_Expected Annual Damage',
       'A.2_Dike Investment Costs', 'A.2_Expected Number of Deaths',
       'A.3_Expected Annual Damage', 'A.3_Dike Investment Costs',
       'A.3_Expected Number of Deaths', 'A.4_Expected Annual Damage',
       'A.4_Dike Investment Costs', 'A.4_Expected Number of Deaths',
       'A.5_Expected Annual Damage', 'A.5_Dike Investment Costs',
       'A.5_Expected Number of Deaths', 'RfR Total Costs',
       'Expected Evacuation Costs'],
      dtype='object')

# Scenario Selection 

### Choose outcomes of interest 

In [10]:
#determine thresholds 
max_expected_annual_damage = 1
max_dike_investement = 10000
max_number_deaths = 1 
max_rfr_costs = 200000
max_evacuation_costs = 5000



In [11]:
outcomes[['A.4_Expected Annual Damage','A.4_Dike Investment Costs', 'A.4_Expected Number of Deaths',
          'RfR Total Costs','Expected Evacuation Costs']]

outcomes_of_interest = outcomes[(outcomes['A.4_Expected Annual Damage']< max_expected_annual_damage) 
                                &( outcomes['Expected Evacuation Costs']<max_evacuation_costs)]

y = (outcomes['A.4_Expected Annual Damage']< max_expected_annual_damage)&( outcomes['Expected Evacuation Costs']<max_evacuation_costs)
#y

In [12]:
outcomes_of_interest.head()
#shape(outcomes_of_interest) 

,A.1_Expected Annual Damage,A.1_Dike Investment Costs,A.1_Expected Number of Deaths,A.2_Expected Annual Damage,A.2_Dike Investment Costs,A.2_Expected Number of Deaths,A.3_Expected Annual Damage,A.3_Dike Investment Costs,A.3_Expected Number of Deaths,A.4_Expected Annual Damage,A.4_Dike Investment Costs,A.4_Expected Number of Deaths,A.5_Expected Annual Damage,A.5_Dike Investment Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,0.0,1.908790e+08,0.0,0.0,2.974292e+08,0.0,0.0,1.332576e+08,0.0,0.0,2.986641e+07,0.0,0.000000e+00,7.908444e+07,0.000000,1.208900e+09,0.000000
1,0.0,1.908790e+08,0.0,0.0,2.974292e+08,0.0,0.0,1.332576e+08,0.0,0.0,2.986641e+07,0.0,2.225764e+06,7.908444e+07,0.000234,1.208900e+09,138.398562
3,0.0,1.908790e+08,0.0,0.0,2.974292e+08,0.0,0.0,1.332576e+08,0.0,0.0,2.986641e+07,0.0,0.000000e+00,7.908444e+07,0.000000,1.208900e+09,0.000000
5,0.0,1.908790e+08,0.0,0.0,2.974292e+08,0.0,0.0,1.332576e+08,0.0,0.0,2.986641e+07,0.0,2.182518e+07,7.908444e+07,0.001740,1.208900e+09,1068.039337
6,0.0,1.908790e+08,0.0,0.0,2.974292e+08,0.0,0.0,1.332576e+08,0.0,0.0,2.986641e+07,0.0,1.163788e+07,7.908444e+07,0.001209,1.208900e+09,733.561547


In [13]:
experiments.head()

,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,113,150.062951,1.5,0.395811,215.256276,10,0.113834,248.433848,1.0,0.519592,...,2,1,7,0,7,2,3,5,0,dikesnet
1,36,166.436369,10,0.523246,104.413735,1.0,0.328583,128.462235,10,0.078784,...,2,1,7,0,7,2,3,6,0,dikesnet
2,120,345.096128,1.5,0.488484,224.018908,10,0.946071,211.803491,10,0.806133,...,2,1,7,0,7,2,3,7,0,dikesnet
3,26,120.907419,1.5,0.678298,257.401605,10,0.818209,316.620822,10,0.460272,...,2,1,7,0,7,2,3,8,0,dikesnet
4,1,208.520733,1.0,0.777740,48.623277,10,0.515623,171.997295,1.5,0.783059,...,2,1,7,0,7,2,3,9,0,dikesnet


In [14]:
experiments_of_interest = experiments.loc[y]
outcomes_df = pd.DataFrame({k:v[y] for k,v in outcomes.items()})

#normalize outcomes on unit interval to ensure equal weighting of outcomes
x = outcomes_df.values 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
normalized_outcomes = pd.DataFrame(x_scaled, columns=outcomes_df.columns)

In [15]:
#Question: This cell doesn't run with a large dataframe as input for the outcomes and experiments. 
## How can we fix this? 

# foutmelding: 
#The kernel for Documents/GitHub/EPA1361_group16/MORDM.ipynb appears to have died. It will restart automatically. 

n_scen = experiments.loc[y].shape[0]
indices = range(n_scen)
set_size = 5

n_scen
combinations = itertools.combinations(indices, set_size)
combinations = list(combinations)

In [16]:
#combinations
print(len(combinations))

324632


In [17]:
sampled_combinations = random.sample(combinations, 100)

In [18]:
distances = squareform(pdist(normalized_outcomes.values))

cores = os.cpu_count()
partial_function = functools.partial(find_maxdiverse_scenarios, distances)


#Difference ProcessPool en ThreadPool 
# https://superfastpython.com/threadpoolexecutor-vs-processpoolexecutor/#Comparison_of_ThreadPoolExecutor_vs_ProcessPoolExecutor
#with ProcessPoolExecutor(max_workers=cores) as executor:
with ThreadPoolExecutor(max_workers=cores) as executor:
    worker_data = np.array_split(combinations, cores)
    results = [e for e in executor.map(partial_function, worker_data)]
    results = list(itertools.chain.from_iterable(results))

In [19]:
results.sort(key=lambda entry:entry[0], reverse=True)
most_diverse = results[0]
most_diverse

([1.9051127644260117], array([ 0,  9, 23, 27, 31]))

In [20]:
selected = experiments.loc[most_diverse[1], ['A.0_ID flood wave shape', 'A.1_Bmax', 'A.1_Brate', 'A.1_pfail',
       'A.2_Bmax', 'A.2_Brate', 'A.2_pfail', 'A.3_Bmax', 'A.3_Brate',
       'A.3_pfail', 'A.4_Bmax', 'A.4_Brate', 'A.4_pfail', 'A.5_Bmax',
       'A.5_Brate', 'A.5_pfail', 'discount rate 0', 'discount rate 1',
       'discount rate 2', '0_RfR 0', '0_RfR 1', '0_RfR 2', '1_RfR 0',
       '1_RfR 1', '1_RfR 2', '2_RfR 0', '2_RfR 1', '2_RfR 2', '3_RfR 0',
       '3_RfR 1', '3_RfR 2', '4_RfR 0', '4_RfR 1', '4_RfR 2',
       'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2',
       'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2',
       'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2',
       'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2',
       'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2',
       'EWS_DaysToThreat', 'scenario', 'policy', 'model']]
scenarios = [Scenario(f"{index}", **row) for index, row in selected.iterrows()]

In [21]:
# Show what the scenarios entail 

#scenarios

# Mordm Run 

In [22]:
dike_model.outcomes = [ScalarOutcome('A.4_Expected Annual Damage', kind=ScalarOutcome.MINIMIZE),
                       ScalarOutcome('A.4_Dike Investment Costs', kind=ScalarOutcome.MINIMIZE),
                       ScalarOutcome('A.4_Expected Number of Deaths', kind=ScalarOutcome.MINIMIZE),
                       ScalarOutcome('RfR Total Costs', kind=ScalarOutcome.MINIMIZE),
                       ScalarOutcome('Expected Evacuation Costs', kind=ScalarOutcome.MINIMIZE)] 

### Hypervolume determination 

In [23]:
#Question: This cell doesn't work, could you help us? 

ema_logging.log_to_stderr(ema_logging.INFO)

results = []
for scenario in scenarios:
    convergence_metrics = [HyperVolume(minimum=[0,0,0,0,0], maximum=[1000000, 4.20e+07 ,0.02,9.84e+08,1068.45]),
                           EpsilonProgress()]
    epsilons = [0.1,]*len(dike_model.outcomes)
    
    results.append(optimize(scenario, 1e4, dike_model, convergence_metrics, epsilons))

[MainProcess/INFO] pool started with 8 workers
  0%|                                                | 0/10000 [00:00<?, ?it/s]'A.4_Expected Annual Damage'
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/ema_workbench/em_framework/experiment_runner.py", line 92, in run_experiment
    model.run_model(scenario, policy)
  File "/opt/anaconda3/lib/python3.9/site-packages/ema_workbench/util/ema_logging.py", line 152, in wrapper
    res = func(*args, **kwargs)
  File "/opt/anaconda3/lib/python3.9/site-packages/ema_workbench/em_framework/model.py", line 352, in run_model
    self.outcomes_output = outputs
  File "/opt/anaconda3/lib/python3.9/site-packages/ema_workbench/em_framework/model.py", line 67, in outcomes_output
    data = [outputs[var] for var in outcome.variable_name]
  File "/opt/anaconda3/lib/python3.9/site-packages/ema_workbench/em_framework/model.py", line 67, in <listcomp>
    data = [outputs[var] for var in outcome.variable_name]
KeyError: 

KeyboardInterrupt: 

### Epsilon volume determination 

In [25]:
#Question: This cell doesn't work, could you help us? 
ema_logging.log_to_stderr(ema_logging.INFO)

with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.optimize(nfe=5000, epsilons=[0.5, 0.5, 0.5, 0.5,0.5])

Process SpawnPoolWorker-24:
Process SpawnPoolWorker-23:
Process SpawnPoolWorker-25:
Process SpawnPoolWorker-27:
Process SpawnPoolWorker-28:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/anaconda3/lib/python3.9/multiprocessing/queues.py", line 365, in get
    with self._rlock:
  File "/opt/anaconda3/lib/python3.9/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
Traceback (most recent call last):
  File "<string>", line 1, in <module>
Process SpawnPoolWorker-26:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run(

KeyboardInterrupt: 

rocess.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/anaconda3/lib/python3.9/multiprocessing/queues.py", line 365, in get
    with self._rlock:
  File "/opt/anaconda3/lib/python3.9/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/opt/anaconda3/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
Process SpawnPoolWorker-22:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/anaconda3/lib/python3.9/multiprocessing/queues.py", line 365, in get
    wit

In [28]:
ema_logging.log_to_stderr(ema_logging.INFO)

with SequentialEvaluator(dike_model) as evaluator:
    results1 = evaluator.optimize(nfe=5e3, searchover='levers',
                                 epsilons=[0.1,]*len(dike_model.outcomes))




  0%|                                                 | 0/5000 [00:00<?, ?it/s][MainProcess/ERROR] 'ID flood wave shape'
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/ema_workbench/em_framework/experiment_runner.py", line 92, in run_experiment
    model.run_model(scenario, policy)
  File "/opt/anaconda3/lib/python3.9/site-packages/ema_workbench/util/ema_logging.py", line 152, in wrapper
    res = func(*args, **kwargs)
  File "/opt/anaconda3/lib/python3.9/site-packages/ema_workbench/em_framework/model.py", line 350, in run_model
    outputs = self.run_experiment(experiment)
  File "/opt/anaconda3/lib/python3.9/site-packages/ema_workbench/util/ema_logging.py", line 152, in wrapper
    res = func(*args, **kwargs)
  File "/opt/anaconda3/lib/python3.9/site-packages/ema_workbench/em_framework/model.py", line 403, in run_experiment
    model_output = self.function(**experiment)
  File "/Users/anne-keedoing/Documents/GitHub/EPA1361_group16/dike_model_

EMAError: exception in run_model
Caused by: KeyError: 'ID flood wave shape'

## Plots 

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2)
for i, (_, convergence) in enumerate(results):
    ax1.plot(convergence.nfe, convergence.hypervolume, label=f'scenario {i}')
    ax2.plot(convergence.nfe, convergence.epsilon_progress, label=f'scenario {i}')

ax1.set_ylabel('hypervolume')
ax1.set_xlabel('nfe')
ax2.set_ylabel('$\epsilon$ progress')
ax2.set_xlabel('nfe')
fig.legend()
plt.show()

In [ ]:
colors = iter(sns.color_palette())

data = results[0][0].iloc[:, 5::]
limits = parcoords.get_limits(data)

limits.loc[0, ['inertia', 'reliability']] = 1
limits.loc[0, 'max_P'] = 4 # max over results based on quick inspection not shown here
limits.loc[0, 'utility'] = 1 # max over results based on quick inspection not shown here
limits.loc[1, :] = 0
paraxes = parcoords.ParallelAxes(limits)


for i, (result, _) in enumerate(results):
    color = next(colors)
    data = result.iloc[:, 5::]
    paraxes.plot(data, label=f'scenario {i}', color=color)

paraxes.legend()
plt.show()



## Deep Uncertainty 

In [ ]:
#specify the policies 

policies = []
for i, (result, _) in enumerate(results):
    result = result.iloc[:, 0:5]
    for j, row in result.iterrows():
        policy = Policy(f'scenario {i} option {j}', **row.to_dict())
        policies.append(policy)
        
## Our way: 

#Identify policies based on open exploration 

policies = [Policy('policy 1', **{'0_RfR 0':1,
                                  '0_RfR 1':1,
                                  '0_RfR 2':1,
                                  'A.1_DikeIncrease 0':5,
                                 'A.1_DikeIncrease 1':5}),
           Policy('policy 2', **{'4_RfR 0':1,
                                  '4_RfR 1':1,
                                  '4_RfR 2':1,
                                  'A.5_DikeIncrease 0':5}),
           Policy('policy 3', **{'1_RfR 0':1,
                                  '2_RfR 1':1,
                                  '3_RfR 2':1,
                                  'A.3_DikeIncrease 0':5})]
        

In [ ]:
with MultiprocessingEvaluator(model) as evaluator:
    reeevaluation_results = evaluator.perform_experiments(1000, policies=policies)

Calculate both the maximum regret, and the domain criterion using the values provided in [Bartholomew and Kwakkel (2020)](https://doi.org/10.1016/j.envsoft.2020.104699). Ignore the max_P objective.



In [ ]:

experiments, outcomes = reeevaluation_results

#as defined earlier: 
thresholds = {'utility':0.75, 'inertia':0.99, 'reliability':0.8}

overall_scores = {}
for policy in experiments.policy.unique():
    logical = experiments.policy == policy
    scores = {}
    for k, v in outcomes.items():
        try:
            n = np.sum(v[logical]>=thresholds[k])
        except KeyError:
            continue
        scores[k] = n/1000 
    overall_scores[policy] = scores
        
overall_scores = pd.DataFrame(overall_scores).T

In [ ]:
limits = parcoords.get_limits(overall_scores)
paraxes = parcoords.ParallelAxes(limits)
paraxes.plot(overall_scores)
plt.show()

Alternative way?? 

In [ ]:
overall_scores = {}
regret = []
for scenario in experiments.scenario.unique():
    logical = experiments.scenario==scenario
    temp_results = {k:v[logical] for k,v in outcomes.items()}
    temp_results = pd.DataFrame(temp_results)
    temp_experiments = experiments[experiments.scenario==scenario]
        
    best = temp_results.max()
    best['max_P'] = temp_results.max_P.min()
    scenario_regret = a - best
    scenario_regret['policy'] = temp_experiments.policy.values    
    regret.append(scenario_regret)



In [ ]:
regret = pd.concat(regret)
maxregret = regret.groupby('policy').max()

In [ ]:
limits = parcoords.get_limits(maxregret)
paraxes = parcoords.ParallelAxes(maxregret)
paraxes.plot(maxregret)
plt.show()

## Notes 

In [ ]:
outcomes[['A.4_Expected Annual Damage','A.4_Dike Investment Costs', 'A.4_Expected Number of Deaths',
          'RfR Total Costs','Expected Evacuation Costs']]

In [ ]:
levers = ['A.0_ID flood wave shape', 
        'A.1_Bmax', 'A.1_Brate', 'A.1_pfail',
        'A.2_Bmax', 'A.2_Brate', 'A.2_pfail', 
        'A.3_Bmax', 'A.3_Brate', 'A.3_pfail', 
        'A.4_Bmax', 'A.4_Brate', 'A.4_pfail', 
        'A.5_Bmax', 'A.5_Brate', 'A.5_pfail', 
          'discount rate 0', 'discount rate 1','discount rate 2', 
          '0_RfR 0', '0_RfR 1', '0_RfR 2', 
          '1_RfR 0','1_RfR 1', '1_RfR 2', 
          '2_RfR 0', '2_RfR 1', '2_RfR 2', 
          '3_RfR 0','3_RfR 1', '3_RfR 2', 
          '4_RfR 0', '4_RfR 1', '4_RfR 2',
       'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2',
       'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2',
       'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2',
       'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2',
       'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2',
       'EWS_DaysToThreat', 'scenario', 'policy', 'model']



levers_of_interest = ['A.4_Bmax', 'A.4_Brate', 'A.4_pfail',
        '0_RfR 0', '0_RfR 1', '0_RfR 2', 
        '1_RfR 0','1_RfR 1', '1_RfR 2', 
        '2_RfR 0','2_RfR 1', '2_RfR 2', 
        '3_RfR 0','3_RfR 1', '3_RfR 2', 
        '4_RfR 0','4_RfR 1', '4_RfR 2',
       'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2',
       'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2',
       'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2',
       'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2',
       'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2']

names_outcomes = ['A.1_Expected Annual Damage', 'A.1_Dike Investment Costs',
       'A.1_Expected Number of Deaths', 'A.2_Expected Annual Damage',
       'A.2_Dike Investment Costs', 'A.2_Expected Number of Deaths',
       'A.3_Expected Annual Damage', 'A.3_Dike Investment Costs',
       'A.3_Expected Number of Deaths', 'A.4_Expected Annual Damage',
       'A.4_Dike Investment Costs', 'A.4_Expected Number of Deaths',
       'A.5_Expected Annual Damage', 'A.5_Dike Investment Costs',
       'A.5_Expected Number of Deaths', 'RfR Total Costs',
       'Expected Evacuation Costs']

In [ ]:
#Identify policies based on open exploration 

# policies = [Policy('policy 1', **{'A.0_ID flood wave shape':, 'A.1_Bmax', 'A.1_Brate', 'A.1_pfail',
#        'A.2_Bmax', 'A.2_Brate', 'A.2_pfail', 'A.3_Bmax', 'A.3_Brate',
#        'A.3_pfail', 'A.4_Bmax', 'A.4_Brate', 'A.4_pfail', 'A.5_Bmax',
#        'A.5_Brate', 'A.5_pfail', 'discount rate 0', 'discount rate 1',
#        'discount rate 2', '0_RfR 0', '0_RfR 1', '0_RfR 2', '1_RfR 0',
#        '1_RfR 1', '1_RfR 2', '2_RfR 0', '2_RfR 1', '2_RfR 2', '3_RfR 0',
#        '3_RfR 1', '3_RfR 2', '4_RfR 0', '4_RfR 1', '4_RfR 2',
#        'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2',
#        'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2',
#        'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2',
#        'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2',
#        'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2',
#        'EWS_DaysToThreat', 'scenario', 'policy', 'model'})]


#            Policy('policy 2', **{'4_RfR 0':1,
#                                   '4_RfR 1':1,
#                                   '4_RfR 2':1,
#                                   'A.5_DikeIncrease 0':5}),
#            Policy('policy 3', **{'1_RfR 0':1,
#                                   '2_RfR 1':1,
#                                   '3_RfR 2':1,
#                                   'A.3_DikeIncrease 0':5})]


